<a href="https://colab.research.google.com/github/jazzminegarg/Quantum-Computing/blob/main/Assigments/Assignment9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit==0.43.2 qiskit-algorithms

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of qiskit-algorithms to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.5/241.5 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.9/306.9 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 17.7 MB/s eta 0:00:00


In [ ]:
import math
import random
import itertools
from fractions import Fraction
from qiskit import BasicAer, execute
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.tools.visualization import plot_histogram

def find_period_classical(x, N):
    n = 1
    t = x
    while t != 1:
        t *= x
        t %= N
        n += 1
    return n

def continued_fractions(x, y):
    while y:
        x, y = y, x % y
    return x

def circuit_amod15(qc, qr, cr, a):
    if a == 2:
        qc.cswap(qr[4], qr[3], qr[2])
        qc.cswap(qr[4], qr[2], qr[1])
        qc.cswap(qr[4], qr[1], qr[0])
    elif a == 7:
        qc.cswap(qr[4], qr[1], qr[0])
        qc.cswap(qr[4], qr[2], qr[1])
        qc.cswap(qr[4], qr[3], qr[2])
        qc.cx(qr[4], qr[3])
        qc.cx(qr[4], qr[2])
        qc.cx(qr[4], qr[1])
        qc.cx(qr[4], qr[0])
    elif a == 8:
        qc.cswap(qr[4], qr[1], qr[0])
        qc.cswap(qr[4], qr[2], qr[1])
        qc.cswap(qr[4], qr[3], qr[2])
    elif a == 11:
        qc.cswap(qr[4], qr[2], qr[0])
        qc.cswap(qr[4], qr[3], qr[1])
        qc.cx(qr[4], qr[3])
        qc.cx(qr[4], qr[2])
        qc.cx(qr[4], qr[1])
        qc.cx(qr[4], qr[0])
    elif a == 13:
        qc.cswap(qr[4], qr[3], qr[2])
        qc.cswap(qr[4], qr[2], qr[1])
        qc.cswap(qr[4], qr[1], qr[0])
        qc.cx(qr[4], qr[3])
        qc.cx(qr[4], qr[2])
        qc.cx(qr[4], qr[1])
        qc.cx(qr[4], qr[0])

def circuit_aperiod15(qc, qr, cr, a):
    if a == 11:
        circuit_11period15(qc, qr, cr)
        return
    qc.x(qr[0])
    qc.h(qr[4])
    qc.h(qr[4])
    qc.measure(qr[4], cr[0])
    qc.reset(qr[4])
    qc.h(qr[4])
    qc.cx(qr[4], qr[2])
    qc.cx(qr[4], qr[0])
    qc.u1(math.pi / 2., qr[4]).c_if(cr, 1)
    qc.h(qr[4])
    qc.measure(qr[4], cr[1])
    qc.reset(qr[4])
    qc.h(qr[4])
    circuit_amod15(qc, qr, cr, a)
    qc.u1(3. * math.pi / 4., qr[4]).c_if(cr, 3)
    qc.u1(math.pi / 2., qr[4]).c_if(cr, 2)
    qc.u1(math.pi / 4., qr[4]).c_if(cr, 1)
    qc.h(qr[4])
    qc.measure(qr[4], cr[2])

def circuit_11period15(qc, qr, cr):
    qc.x(qr[0])
    qc.h(qr[4])
    qc.h(qr[4])
    qc.measure(qr[4], cr[0])
    qc.reset(qr[4])
    qc.h(qr[4])
    qc.h(qr[4])
    qc.measure(qr[4], cr[1])
    qc.reset(qr[4])
    qc.h(qr[4])
    circuit_amod15(qc, qr, cr, 11)
    qc.p(math.pi / 4., qr[4]).c_if(cr, 3)
    qc.p(math.pi / 2., qr[4]).c_if(cr, 2)
    qc.p(3. * math.pi / 4., qr[4]).c_if(cr, 1)
    qc.h(qr[4])
    qc.measure(qr[4], cr[2])

def shors_algorithm(N, x):
    t = 9
    qr = QuantumRegister(5, 'qr')
    cr = ClassicalRegister(3, 'cr')
    qc = QuantumCircuit(qr, cr)
    qc.x(qr[0])
    circuit_aperiod15(qc, qr, cr, x)
    backend = BasicAer.get_backend('qasm_simulator')
    job = execute(qc, backend, shots=8192)
    result = job.result()
    counts = result.get_counts(qc)
    plot_histogram(counts)
    T = 2**t
    for z, count in counts.items():
        if count > 10:
            frac = Fraction(int(z, 2), T)
            r = frac.denominator
            if r % 2 == 0:
                p = math.gcd(x**(r//2) + 1, N)
                q = math.gcd(x**(r//2) - 1, N)
                if p != 1 and q != 1:
                    return p, q
    return None, None

N = 4
x = 11
p, q = shors_algorithm(N, x)

if p is not None and q is not None:
    print(f"Factors found for N = {N}: {p}, {q}")
else:
    print("Factors not found. Try again with a different coprime integer.")

Factors found for N = 4: 2, 4


In [ ]:
def initialize_s(qc, qubits):
    """Apply a H-gate to 'qubits' in qc"""
    for q in qubits:
        qc.h(q)
    return qc

def diffuser(nqubits):
    qc = QuantumCircuit(nqubits)
    # Apply transformation |s> -> |00..0> (H-gates)
    for qubit in range(nqubits):
        qc.h(qubit)
    # Apply transformation |00..0> -> |11..1> (X-gates)
    for qubit in range(nqubits):
        qc.x(qubit)
    # Do multi-controlled-Z gate
    qc.h(nqubits-1)
    qc.mct(list(range(nqubits-1)), nqubits-1)  # multi-controlled-toffoli
    qc.h(nqubits-1)
    # Apply transformation |11..1> -> |00..0>
    for qubit in range(nqubits):
        qc.x(qubit)
    # Apply transformation |00..0> -> |s>
    for qubit in range(nqubits):
        qc.h(qubit)
    # We will return the diffuser as a gate
    U_s = qc.to_gate()
    U_s.name = "U$_s$"
    return U_s

def oracle(qc, element, nqubits):
    """
    Marks the element to search by applying an X gate
    to the output qubit if the input qubits represent the element.
    """
    # Convert the element to binary representation
    element_binary = bin(element)[2:].zfill(nqubits)

    # Apply X gates to qubits corresponding to 1s in the element's binary
    for i, bit in enumerate(element_binary):
        if bit == '1':
            qc.x(i)

    # Apply a multi-controlled-Z gate to mark the element
    qc.h(nqubits - 1) # Ancilla qubit for phase kickback
    qc.mct(list(range(nqubits - 1)), nqubits - 1)
    qc.h(nqubits - 1)

    # Uncompute the X gates to restore the input state
    for i, bit in enumerate(element_binary):
        if bit == '1':
            qc.x(i)

    # Return the oracle as a gate
    oracle_gate = qc.to_gate()
    oracle_gate.name = "Oracle"
    return oracle_gate

def grover_search(element, list_size):
    """
    Performs Grover's search to find the given element in a list of size list_size.
    """
    # Calculate the number of qubits needed
    nqubits = int(math.log2(list_size)) + 1  # +1 for ancilla qubit

    # Create a quantum circuit
    qc = QuantumCircuit(nqubits, nqubits - 1)

    # Initialize the qubits to the superposition state
    qc = initialize_s(qc, range(nqubits - 1))
    qc.h(nqubits - 1) # Ancilla qubit initialization

    # Calculate the optimal number of iterations
    iterations = int(round(math.pi / 4 * math.sqrt(list_size)))

    # Apply the Grover iteration (oracle and diffuser) the calculated number of times
    for _ in range(iterations):
        qc.append(oracle(QuantumCircuit(nqubits), element, nqubits), range(nqubits))
        qc.append(diffuser(nqubits-1), range(nqubits-1))

    # Measure the qubits
    qc.measure(range(nqubits - 1), range(nqubits - 1))

    # Execute the circuit on a simulator
    backend = Aer.get_backend('qasm_simulator')
    job = execute(qc, backend, shots=1024)
    result = job.result()
    counts = result.get_counts()

    # Return the most frequent measurement result (the searched element)
    max_result = max(counts, key=counts.get)
    return int(max_result, 2)

# Example usage:
element_to_search = 5 # Element we are looking for in the list
list_size = 8       # Number of elements in the list (should be a power of 2)

result = grover_search(element_to_search, list_size)
print(f"Found element: {result}")

Found element: 5
